# Text classification using a Naive Bayes classifier

Email messages can be classified as _Spam_ or _Ham_ using a simple naive bayes classifier using a _Bag of Words_ model.

## Reading data

The training data can be downloaded from the [TREC 07 spam corpus](http://plg.uwaterloo.ca/~gvcormac/treccorpus07/).
It consists of around 75K email messages tagged as spam or ham and their distribution is about 2/3 Spam and 1/3 Ham.

### Parsing the messages

For this notebook, we will consider only subjects and text content. The data needs to be preprocessed for the sake of dimentionality reduction by doing:

* Casefolding
* URL normalization
* Stop word filtering
* Porter stemming

A snippet from `message.py` showing the code implementation can be read:

```python
def processedText(self):
    ''' Stems and filters stop words, among other things '''

    try:
        # Concatenate subject and body
        txt = self.subject.lower() + '\n' + self.text.lower()
        # Replace URLs for a token
        txt = url.sub('-URL-', txt)
        # Tokenize
        tokens = nltk.word_tokenize(txt)
        # Stem and remove stop words
        stems = [self.stemmer.stem(w) for w in tokens if w not in stopwords]
    except:
        stems = []

    return ' '.join(stems)
```


In [1]:
import glob, os
import sklearn as sk
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from message import Message

DATA_DIR='data/data'
LABELS_PATH='data/partial/index'

In [ ]:
# Read messages
print "Reading messages ..."
start = datetime.now()

messages = []
heldout_messages = {}
for path in glob.iglob(os.path.join(DATA_DIR, '*')):
    m = Message.load(path)
    messages.append(m)

# Read the labels
print "Reading labels ..."

with open(LABELS_PATH) as f:
    labels_dict = {}
    for line in f:
        line = line.strip()
        label, key = line.split()
        labels_dict[key.split('/')[-1]] = 1 if label.lower() == 'ham' else 0
        
end = datetime.now()

print "%i seconds reading and parsing ..." % (end - start).seconds

## The bag of words model
A text is represented as a multiset containing its words disregarding their order. Imagine [dumping all the words in a document into a bag](https://en.wikipedia.org/wiki/Bag-of-words_model). Despite losing syntax and structure, this representation gives good results for this task.

### Vector representation
Each bag of words is represented by an integer vector whose dimentionality is the _size of the vocabulary_ and each entry contains the _term frequency_ for a given word in the message's vector. This yields very sparse vectors, because each message contains a very small subset of the words in the vocabulary, thus, _the majority of the entries will be zeroes_.

$$\mathbf{d}_i = [0, 0, 2, ..., 0, 5, 0]$$

In [ ]:
# Split the data in 70% training, 30% testing
print "Creating feature vectors ..."
start = datetime.now()

vectorizer = sk.feature_extraction.text.CountVectorizer()
X = vectorizer.fit_transform([m.processedText for m in messages])
y = np.array([labels_dict[m.id] for m in messages])

end = datetime.now()

print "%i seconds extracting features ..." % (end - start).seconds

## The Naive Bayes classifier

The intiution behind _Naive Bayes_ is to find the probability of each class, in this case __Spam__ and __Ham__ given a message and select whichever class is more likely:

$$ \hat{C} = \arg\max_C P(C \mid \mathbf{d}_i)$$

We use _Baye's rule_ to turn the probability around:

$$P(C \mid \mathbf{d}_i) = P(\mathbf{d}_i \mid C)\frac{P(C)}{P(\mathbf{d}_i)}$$

Since we are not concerned with the extact probability of each class, we can dismiss the denominator and the maximum of the resulting likelihood will happen at the same point as with the probabilities:

$$P(C \mid \mathbf{d}_i) \propto P(\mathbf{d}_i \mid C)P(C)$$

## Naive assumption

To simplify the model, we assume _conditional independence_ of the words in a document, given the class. This is the naive assumption of the model, which reduces the number of estimated parameters to a linear function of the size vocabulary. The new likelihood formula is:

$$P(C \mid \mathbf{d}_i) \propto \prod_{j = 1}^{|V|}P(d_{ij}\mid C)P(C)$$

## Multinomial distribution and out-of-vocabulary words

Our feature vectors contain term frequencies, which are integer numbers. A term's conditional pobability distribution can be naturaly modeled by a [multinomial distribution](https://en.wikipedia.org/wiki/Multinomial_distribution). The parameters are estimated using _Maximum Likelihood Estimation_, which in this case is counting the ocurrences of each team across the messages of a given class _C_. The prior's (Spam or Ham) marginal distribution is computed the same way.

It is possible that a message contains a word not seen during training. This would break the product, as it's probability would be zero and drag down the whole computation. To handle this, a new term representing the _OOV_ words is introduced and a small piece of probability mass is transfered to it. This is known as [Additive Smoothing](add one smoothing trigram) and guarantees that no term in the product will ever be zero, making the computation feasible.

## Dealing with numerical _underflow_

By definition, probabilities range from 0 to 1, and the joint probability of a message given a class will be a _very tiny number_ resulting from multiplying tens of thousands or probably hundreds of thousands of individual term conditional probabilities. Theoretically there's no problem with this, but computers can't handle such small quantities with their floating point representation because they become _zero_ very quickly.

To handle this, we instead maximize _log probabilities_, and our target maximization becomes:

$$ \hat{C} = \arg\max_C \log P(C \mid \mathbf{d}_i)$$

$$\log P(C \mid \mathbf{d}_i) \propto \log P(\mathbf{d}_i \mid C) + \log P(C)$$

$$\log P(C \mid \mathbf{d}_i) \propto \sum_{j = 1}^{|V|} \log P(d_{ij}\mid C) + \log P(C)$$

Maximizing this expression gives the same result as maximizing the original because the logarithm is a [monotonic function](https://en.wikipedia.org/wiki/Monotonic_function).

# Implementing NB with code

Fortunately, all these computations have been already implemented by [Scikit Learn](http://scikit-learn.org/stable/modules/naive_bayes.html) and all we need to do is write a couple lines of code!

In [ ]:
# Split the data set in training 70% and testing 30%
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.7)

print "Training ..."
start = datetime.now()
nb = MultinomialNB()
nb.fit(X_train, y_train)
end = datetime.now()

In [ ]:
print "Testing:"
y_pred = nb.predict(X_test)
print classification_report(y_test, y_pred, target_names=['Spam', 'Ham'])
print
print "Accuracy: %.3f" % accuracy_score(y_test, y_pred)
print
print "Confusion matrix:"
print confusion_matrix(y_test, y_pred)

# Links
* [More on text classification](http://nlp.stanford.edu/IR-book/html/htmledition/text-classification-and-naive-bayes-1.html)
* [Wikipedia entry for Naive Bayes](https://en.wikipedia.org/wiki/Naive_Bayes_classifier)
* [Scikit Learn](http://scikit-learn.org/)
* [NLTK](http://www.nltk.org)